In [144]:
from gensim.parsing.preprocessing import stem_text
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import corpora, models 
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
sys.path.append('../')
from utils import word_frequency_per_player, remove_similar_rows_per_player
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report
import pyLDAvis
from pyLDAvis.gensim_models import prepare
import re
from seaborn import countplot, distplot, heatmap, histplot
import numpy as np




# Load data and convert to stemmed and BERT data 

In [145]:
# load data and drop NA's
df = pd.read_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Preprocessing/data_clean/de_clean_3.csv')

# Because Bayer Leverkusen and Leverkusen appeared in every single cluster we remove it 

In [146]:
df['data'] = df['data'].apply(lambda x: re.sub(r"\b(bayerleverkusen)\b", '', str(x)))
df['data'] = df['data'].apply(lambda x: re.sub(r"\b(leverkusen)\b", '', str(x)))

# Stemming and Vectorization

In [147]:
# create two copies 
df_stem = df.copy()

# create stemmed data
df_stem['data'] = df_stem['data'].apply(stem_text)

df_stem

,data,player,language,publishedAt
0,trainer alonso mitgereisten fan monaco glucksmoment hebt tor euro highlight geht bundesliga sonntag uhr freiburg weiter,exequiel palacios,de,2023-02-24T09:33:31Z
1,sechser droht viereinhalb monaten paus bitter abschi dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden amiri demirbai andrich voraussetzungen extrem schwierig fur aranguiz zuletzt aufgrund personellen engpass innenverteidigung zentral abwehrkett eingesetzt andrich kunftig mittelfeld auflaufen durft alonso vorrucken jahrigen sech sonntag hertha bsc nicht sicher einstufen mocht,exequiel palacios,de,2023-03-03T21:35:13Z
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfin hinspiel europaleagu ferencvaro budapest empfangt andrich fehlen sah defens mittelfeldspiel erfolg elfmeterschiessen monaco dritt gelb kart gesperrt massiv schwachung fur werkself doppelsech andrich zuletzt erfolgsfaktor andrich ausgerechnet kurzen phase saison gleichzeitig spielerisch uberzeugt erfolgreichen fussbal spielt zwangspaus einlegen muss wurmt bedingt da argerlich irgendwann ware sperr wahrscheinlich gekommen daher lieber jetzt eventuel viertel halbfinal kein sorgen erklart jahrig lacheln,exequiel palacios,de,2023-03-07T11:34:39Z
3,bakker mittwoch abschluss train nicht ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sech verzichten muss andrich gesperrt fur stehen demirbai amiri bereit frimpong budapest einsatzbereit hertha wegen muskular problem bereit minuten platz verlassen mussen gut trainiert fuhlt besser dabei erklart alonso grossen respekt ungarn hat da sehr guter gegner sehe niveau monaco,exequiel palacios,de,2023-03-08T14:25:18Z
4,aktuel nachrichten politik wirtschaft sport kultur allerd andrich argentinischen weltmeist beid eigentlich gesetzten sechser gesperrt charl aranguiz zudem verletzt donnerstag machten sach zumindest ordentlich mehr thema,exequiel palacios,de,2023-03-09T19:53:46Z
...,...,...,...,...
617,wehrt europaleagu lang gut schwere hypothek fur ruckspiel romischen fan lautstark hymn choreo uber rang knallkorpern kurz spiel fur fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp alonso mourinho begrussten spiel herzlichen umarmung sobald spiel begann spielt interessant duell beiden spielfeldrand alonso dirigiert tigert coach zone standard setzt bank uberliess einteilung assistenten sebastian parrilla mourinho verfolgt geschehen meist handen taschen immer lief zuruck bank besprach assistenten nicht kamen plotzlich wutanfal anfeuerungsschrei herau immer versucht mourinho team pushen gegner unparteiischen konzept bringen,piero hincapie,de,2023-05-12T10:02:12Z
618,alonso taktik duell alten lehrmeist mourinho niederlag einstecken mussen droht einzug endspiel europaleagu verpassen halbfin hinspiel rom steht alonso trainiert werkself ruckspiel woch ganz schweren aufgab dritt europacup final vereinsgeschicht erreichen titel seit traumen romischen fan lautstark hymn choreo uber rang knallkorpern kurz spiel fur fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp alonso mourinho begrussten spiel herzlichen umarmung sobald spiel begann spielt interessant duell beiden spielfeldrand alonso dirigiert tigert coach zone standard setzt bank uberliess einteilung assistenten sebastian parrilla mourinho verfolgt geschehen meist handen taschen i

In [148]:
tfidf = TfidfVectorizer(max_df=0.95, min_df=0.02)
df_tfidf = tfidf.fit_transform(df_stem.data).toarray()

In [149]:
df_tfidf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [150]:
#Store the frequency matrix in data_tfidf. 
data_tfidf = pd.DataFrame(df_tfidf, columns=tfidf.vocabulary_.keys())

#data_tfidf['target_names']=sorted_df_ng.target_names.values #we can't use that because otherwise knn wouldn't work
data_tfidf.head()

,trainer,alonso,fan,monaco,tor,euro,geht,bundesliga,sonntag,uhr,freiburg,weiter,sechser,droht,paus,dafur,lang,zudem,seit,woch,voll,amiri,demirbai,andrich,extrem,fur,zuletzt,zentral,mittelfeld,durft,jahrigen,sech,hertha,bsc,nicht,sicher,mocht,erklart,budapest,spiel,donnerstag,hinspiel,europaleagu,sah,defens,mittelfeldspiel,erfolg,dritt,gelb,kart,gesperrt,werkself,phase,saison,uberzeugt,fussbal,spielt,muss,da,ware,wahrscheinlich,daher,halbfinal,kein,sorgen,jahrig,bakker,abschluss,train,jedoch,startelf,steht,stehen,bereit,frimpong,wegen,problem,minuten,platz,mussen,gut,trainiert,fuhlt,besser,dabei,grossen,hat,sehr,gegner,niveau,aktuel,nachrichten,politik,wirtschaft,sport,kultur,allerd,weltmeist,beid,eigentlich,verletzt,zumindest,mehr,thema,union,berlin,chancen,deutlich,saint,gillois,stadion,alten,erneut,schwer,kamen,zuschauern,minut,gast,fuhrung,bundesligisten,sein,borussia,dortmund,entscheidenden,moment,gab,zuruck,spielen,mit,weit,auf,sagt,rtl,zweiten,halft,bisschen,gefahrlich,sind,bracht,diesmal,gute,gemacht,lobt,anstel,hatt,gerad,auch,weiss,un,wird,stellt,klub,bayern,munchen,erst,bremen,einfach,niederlag,mainz,funf,positionen,einsatz,beginn,link,konnt,schick,sieht,braucht,vielleicht,zeit,elf,adam,hlozek,adli,zwei,trotz,marvin,gibt,werder,kurz,schluss,per,elfmet,kommt,spat,torschutzen,sich,diabi,feiern,treffer,argentinisch,nachsten,starken,ball,note,niederland,liess,fehlt,munchner,gehen,liga,kimmich,dreht,schiedsricht,tobia,stieler,jeweil,zunachst,gegeben,male,nahm,europacup,dran,zuvor,duell,internationalen,champion,leagu,frankfurt,eintracht,dritten,liegt,zweimal,darauf,foulelfmet,verwandelt,beiden,fallen,karten,wohl,sieg,gebracht,hatten,begann,stark,keeper,sommer,direkt,eck,pariert,verfehlt,distanz,knapp,flank,joao,legt,metern,team,schlug,argentini,einzig,chanc,uberraschend,uber,prozent,ballbesitz,gastgeb,schuss,odilon,kossoun,latt,ein,gelegenheit,trotzdem,wirklich,zufrieden,war,dreifach,wechsel,neben,kam,thoma,der,spieltag,abwesenheit,kommenden,kapitan,zeigt,punkt,bundesligist,viertelfinal,wettbewerb,seri,sieben,pflichtspiel,setzt,boss,moglich,zukunft,kabin,fiel,verstarkten,offensiv,verzog,doppelpass,nachst,ruckstand,ausgleich,hradecki,lief,bvb,entscheidung,tapsoba,hincapi,tah,wirtz,zuschauer,tore,wir,viel,zweikampf,gewonnen,top,gespielt,franzos,foul,strafraum,erstmal,gluck,mannschaft,insgesamt,wahrend,stund,rheinland,meist,traf,fuss,strafstoss,fuhrt,gleich,passiert,schlussphas,torhut,luka,abwehr,stand,april,verloren,halbzeit,verdient,tag,koln,deutschen,verlor,julian,samstag,leipzig,vorbei,rang,vier,schob,trikot,fall,leverkusen,flog,bekam,parti,dazn,nie,bei,deshalb,entschieden,voller,geben,wurd,ganz,sehen,informationen,szene,guten,selten,naturlich,die,besten,all,heut,genau,lasst,nehmen,genug,durften,neue,davon,kann,immer,recht,vollig,klar,worden,ebenfal,kollegen,sogar,hielt,ich,coach,sicht,schnell,wenn,torschutz,zog,gesehen,ebenso,sei,gewesen,aber,rasen,konnen,gross,posit,jung,drei,tage,belastung,wichtig,bleiben,druck,plotzlich,letzten,stuttgart,gewinn,wochen,uberragenden,form,konnten,schalk,spieler,ist,blieb,weiterhin,pass,vfl,wolfsburg,bochum,tempo,eher,wissen,macht,danach,kevin,kaum,pfosten,verteidig,edmond,erfolgreich,sturmer,ging,wirkt,wenig,dennoch,werksklub,vergangenen,intern,richtig,arbeit,fast,rund,halben,jahr,deutschland,damal,vorn,angeschlagen,azmoun,kader,versucht,sardar,schien,vorlag,sekunden,viertelfin,ubernahm,derzeit,gill,dank,ruckspiel,beginnen,aufgab,offen,entscheidend,geschaftsfuhr,werden,erreichen,freitag,nachdem,belgier,vor,leicht,reicht,dfb,aktion,brussel,kampf,halbfin,hoffnung,oberschenkel,erreicht,scheitert,haben,schliesslich,spanier,herau,hohe,gefahr,hofft,final,fruher,schaffen,einzug,nationalspiel,rolf,roll,derweil,sportchef,moglichkeit,dpa,lassen,gesetzt,zusammen,eh,anderlecht,sowi,simon,frei,belgien,zweit,abseit,richtung,flinken,rucken,ziel,paar,langen,profi,sieger,abend,trat,beendet,uefa,cup,niko,kovac,musst,bank,sorgt,verletzten,defensiv,verrichten,mussten,rannten,korperlichkeit,stemmt,dagegen,ko

In [151]:
#Store the feature names in a words list.
words = data_tfidf.columns.tolist()[:-2] 
words

['trainer',
 'alonso',
 'fan',
 'monaco',
 'tor',
 'euro',
 'geht',
 'bundesliga',
 'sonntag',
 'uhr',
 'freiburg',
 'weiter',
 'sechser',
 'droht',
 'paus',
 'dafur',
 'lang',
 'zudem',
 'seit',
 'woch',
 'voll',
 'amiri',
 'demirbai',
 'andrich',
 'extrem',
 'fur',
 'zuletzt',
 'zentral',
 'mittelfeld',
 'durft',
 'jahrigen',
 'sech',
 'hertha',
 'bsc',
 'nicht',
 'sicher',
 'mocht',
 'erklart',
 'budapest',
 'spiel',
 'donnerstag',
 'hinspiel',
 'europaleagu',
 'sah',
 'defens',
 'mittelfeldspiel',
 'erfolg',
 'dritt',
 'gelb',
 'kart',
 'gesperrt',
 'werkself',
 'phase',
 'saison',
 'uberzeugt',
 'fussbal',
 'spielt',
 'muss',
 'da',
 'ware',
 'wahrscheinlich',
 'daher',
 'halbfinal',
 'kein',
 'sorgen',
 'jahrig',
 'bakker',
 'abschluss',
 'train',
 'jedoch',
 'startelf',
 'steht',
 'stehen',
 'bereit',
 'frimpong',
 'wegen',
 'problem',
 'minuten',
 'platz',
 'mussen',
 'gut',
 'trainiert',
 'fuhlt',
 'besser',
 'dabei',
 'grossen',
 'hat',
 'sehr',
 'gegner',
 'niveau',
 'aktuel

In [152]:
kmeans = KMeans(n_clusters = 4, max_iter=1000, random_state=42)
kmeans.fit(data_tfidf)

/opt/homebrew/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(max_iter=1000, n_clusters=4, random_state=42)

In [153]:
common_words = kmeans.cluster_centers_.argsort()[:,-1:-11:-1]

In [154]:
for num, centroid in enumerate(common_words):
    words_in_centroid = [words[word] for word in centroid if word < len(words)]
    print(f"{num}: {', '.join(words_in_centroid)}")
# By adding the if word < len(words) condition, we ensure that only valid indices are used to access the words list or dictionary.

0: adingra, leistungen, sekunden, eck, paus, angeschlagen, nachspielzeit, diesmal, blieb, patricio
1: vfb, nachspielzeit, gewonnen, startelf, nachdem, diesmal, gesperrt, passiert, wenig, dreht
2: eck, angeschlagen, voll, kassiert, richtung, schlusspfiff, wurden, linksverteidig, fur, bedrangt
3: genau, trafen, werksklub, halbfin, gewann, zudem, gewinn, nochmal, eck, angeschlagen


In [155]:
df_stem['cluster'] = kmeans.labels_
df_stem

,data,player,language,publishedAt,cluster
0,trainer alonso mitgereisten fan monaco glucksmoment hebt tor euro highlight geht bundesliga sonntag uhr freiburg weiter,exequiel palacios,de,2023-02-24T09:33:31Z,2
1,sechser droht viereinhalb monaten paus bitter abschi dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden amiri demirbai andrich voraussetzungen extrem schwierig fur aranguiz zuletzt aufgrund personellen engpass innenverteidigung zentral abwehrkett eingesetzt andrich kunftig mittelfeld auflaufen durft alonso vorrucken jahrigen sech sonntag hertha bsc nicht sicher einstufen mocht,exequiel palacios,de,2023-03-03T21:35:13Z,2
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfin hinspiel europaleagu ferencvaro budapest empfangt andrich fehlen sah defens mittelfeldspiel erfolg elfmeterschiessen monaco dritt gelb kart gesperrt massiv schwachung fur werkself doppelsech andrich zuletzt erfolgsfaktor andrich ausgerechnet kurzen phase saison gleichzeitig spielerisch uberzeugt erfolgreichen fussbal spielt zwangspaus einlegen muss wurmt bedingt da argerlich irgendwann ware sperr wahrscheinlich gekommen daher lieber jetzt eventuel viertel halbfinal kein sorgen erklart jahrig lacheln,exequiel palacios,de,2023-03-07T11:34:39Z,2
3,bakker mittwoch abschluss train nicht ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sech verzichten muss andrich gesperrt fur stehen demirbai amiri bereit frimpong budapest einsatzbereit hertha wegen muskular problem bereit minuten platz verlassen mussen gut trainiert fuhlt besser dabei erklart alonso grossen respekt ungarn hat da sehr guter gegner sehe niveau monaco,exequiel palacios,de,2023-03-08T14:25:18Z,2
4,aktuel nachrichten politik wirtschaft sport kultur allerd andrich argentinischen weltmeist beid eigentlich gesetzten sechser gesperrt charl aranguiz zudem verletzt donnerstag machten sach zumindest ordentlich mehr thema,exequiel palacios,de,2023-03-09T19:53:46Z,3
...,...,...,...,...,...
617,wehrt europaleagu lang gut schwere hypothek fur ruckspiel romischen fan lautstark hymn choreo uber rang knallkorpern kurz spiel fur fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp alonso mourinho begrussten spiel herzlichen umarmung sobald spiel begann spielt interessant duell beiden spielfeldrand alonso dirigiert tigert coach zone standard setzt bank uberliess einteilung assistenten sebastian parrilla mourinho verfolgt geschehen meist handen taschen immer lief zuruck bank besprach assistenten nicht kamen plotzlich wutanfal anfeuerungsschrei herau immer versucht mourinho team pushen gegner unparteiischen konzept bringen,piero hincapie,de,2023-05-12T10:02:12Z,0
618,alonso taktik duell alten lehrmeist mourinho niederlag einstecken mussen droht einzug endspiel europaleagu verpassen halbfin hinspiel rom steht alonso trainiert werkself ruckspiel woch ganz schweren aufgab dritt europacup final vereinsgeschicht erreichen titel seit traumen romischen fan lautstark hymn choreo uber rang knallkorpern kurz spiel fur fast einschuchternden rahmen gesorgt zeigt unbeeindruckt beinah blitzstart hingelegt sekunden gespielt andrich hereingab etwa zwolf metern schuss kam zentral halbhoch sodass portugiesisch torhut rui patricio seit donnerstag einsatzen rekordspiel europaleagu wenig muhe hatt chancenlo ware portugiesisch europameist sechsten minut gewesen wirtz verzog knapp alonso mourinho begrussten spiel herzlichen umarmung sobald spiel begann spielt interessant duell beiden spielfeldrand alonso dirigiert tigert coach zone standard setzt bank uberliess einteilung assistenten sebastian parrilla mourinho verfolgt geschehe

In [156]:
clusters = df_stem.groupby(['cluster']).size()
clusters

cluster
0    218
1     30
2    323
3     51
dtype: int64

# LDA Model

In [157]:
# Split the texts into a list of words
texts = [text.split() for text in df_stem['data']]

# Create the Gensim dictionary
dictionary = corpora.Dictionary(texts)

# Remove rare and common words from the dictionary
dictionary.filter_extremes(no_below=118, no_above=0.95)

# Derive the absolute frequency matrix using doc2bow
corpus = [dictionary.doc2bow(text) for text in texts]


In [158]:
lda = models.LdaModel(corpus, num_topics=4, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)


In [159]:
lda.show_topics()

[(0,
  '0.321*"fur" + 0.206*"nicht" + 0.148*"spiel" + 0.081*"leverkusen" + 0.078*"adli" + 0.075*"uber" + 0.065*"mehr" + 0.024*"sagt" + 0.000*"europaleagu" + 0.000*"alonso"'),
 (1,
  '0.218*"frimpong" + 0.217*"tor" + 0.173*"diabi" + 0.148*"ball" + 0.124*"hlozek" + 0.084*"uber" + 0.032*"mehr" + 0.000*"adli" + 0.000*"leverkusen" + 0.000*"hradecki"'),
 (2,
  '0.520*"alonso" + 0.242*"trainer" + 0.119*"hradecki" + 0.088*"bundesliga" + 0.027*"ball" + 0.000*"spiel" + 0.000*"adli" + 0.000*"europaleagu" + 0.000*"uber" + 0.000*"frimpong"'),
 (3,
  '0.325*"wirtz" + 0.302*"europaleagu" + 0.135*"union" + 0.127*"hinspiel" + 0.108*"uber" + 0.000*"spiel" + 0.000*"alonso" + 0.000*"mehr" + 0.000*"leverkusen" + 0.000*"adli"')]

In [160]:
topics=lda[corpus]
df_stem['lda_predicted_clusters']= [max(topics[i],key=lambda item:item[1])[0] for i in range(len(topics))]

In [161]:
df_stem.head()

,data,player,language,publishedAt,cluster,lda_predicted_clusters
0,trainer alonso mitgereisten fan monaco glucksmoment hebt tor euro highlight geht bundesliga sonntag uhr freiburg weiter,exequiel palacios,de,2023-02-24T09:33:31Z,2,2
1,sechser droht viereinhalb monaten paus bitter abschi dafur erscheint verletzungspaus lang zudem konkurrenzsitu doppelsech seit woch voll saft stehenden amiri demirbai andrich voraussetzungen extrem schwierig fur aranguiz zuletzt aufgrund personellen engpass innenverteidigung zentral abwehrkett eingesetzt andrich kunftig mittelfeld auflaufen durft alonso vorrucken jahrigen sech sonntag hertha bsc nicht sicher einstufen mocht,exequiel palacios,de,2023-03-03T21:35:13Z,2,0
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfin hinspiel europaleagu ferencvaro budapest empfangt andrich fehlen sah defens mittelfeldspiel erfolg elfmeterschiessen monaco dritt gelb kart gesperrt massiv schwachung fur werkself doppelsech andrich zuletzt erfolgsfaktor andrich ausgerechnet kurzen phase saison gleichzeitig spielerisch uberzeugt erfolgreichen fussbal spielt zwangspaus einlegen muss wurmt bedingt da argerlich irgendwann ware sperr wahrscheinlich gekommen daher lieber jetzt eventuel viertel halbfinal kein sorgen erklart jahrig lacheln,exequiel palacios,de,2023-03-07T11:34:39Z,2,3
3,bakker mittwoch abschluss train nicht ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sech verzichten muss andrich gesperrt fur stehen demirbai amiri bereit frimpong budapest einsatzbereit hertha wegen muskular problem bereit minuten platz verlassen mussen gut trainiert fuhlt besser dabei erklart alonso grossen respekt ungarn hat da sehr guter gegner sehe niveau monaco,exequiel palacios,de,2023-03-08T14:25:18Z,2,2
4,aktuel nachrichten politik wirtschaft sport kultur allerd andrich argentinischen weltmeist beid eigentlich gesetzten sechser gesperrt charl aranguiz zudem verletzt donnerstag machten sach zumindest ordentlich mehr thema,exequiel palacios,de,2023-03-09T19:53:46Z,3,0


In [162]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(lda, corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.194491  0.240625       1        1  43.325183
1     -0.045307  0.128902       2        1  20.906750
2      0.418293 -0.022318       3        1  18.637482
3     -0.178495 -0.347209       4        1  17.130584, topic_info=           Term        Freq       Total Category  logprob  loglift
0        alonso  537.000000  537.000000  Default  21.0000  21.0000
4           fur  771.000000  771.000000  Default  20.0000  20.0000
19        wirtz  309.000000  309.000000  Default  19.0000  19.0000
6   europaleagu  287.000000  287.000000  Default  18.0000  18.0000
3       trainer  251.000000  251.000000  Default  17.0000  17.0000
..          ...         ...         ...      ...      ...      ...
3       trainer    0.249928  251.098467   Topic4  -8.2426  -5.1481
8         spiel    0.290587  355.730625   Topic4  -8.0918  -5.3457
5         nicht    0.255785  495.532132   Topic4  -8.2194  -5.8047
0        alonso    0.270242  537.679776   Topic4  -8.1644  -5.8314
4           fur    0.255529  771.819811   Topic4  -8.2204  -6.2489

[105 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
13        1  0.994024         adli
0         3  0.998736       alonso
15        2  0.859498         ball
15        3  0.139918         ball
1         3  0.988783   bundesliga
16        2  0.996445        diabi
6         4  0.995965  europaleagu
9         2  0.997187     frimpong
4         1  0.998938          fur
7         4  0.990316     hinspiel
14        2  0.994527       hlozek
18        3  0.996514     hradecki
20        1  0.994916   leverkusen
10        1  0.805795         mehr
10        2  0.191118         mehr
5         1  0.998926        nicht
12        1  0.980507         sagt
8         1  0.997946        spiel
2         2  0.995053          tor
3         3  0.995625      trainer
17        1  0.475377         uber
17        2  0.257386         uber
17        4  0.267892         uber
11        4  0.993243        union
19        4  0.995907        wirtz, R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4])

## Create a second model

In [163]:
lda_2 = models.LdaModel(corpus, num_topics=10, id2word=dictionary, chunksize=10, iterations=100, passes=10, random_state=42)

## Finding the best number of topics

In [164]:
models_arr = {'cluster_' + str(iter): '' for iter in range(2, 15)}
for iter in range(2, 15): 
    models_arr.update({'cluster_'+ str(iter): models.LdaModel(corpus, num_topics=iter, id2word=dictionary, chunksize=150, iterations=100, passes=20, random_state=42)})

In [174]:
models_arr['cluster_4'].show_topics(num_words = 20)

[(0,
  '0.196*"fur" + 0.149*"nicht" + 0.140*"alonso" + 0.125*"spiel" + 0.071*"trainer" + 0.060*"leverkusen" + 0.053*"sagt" + 0.053*"europaleagu" + 0.039*"hradecki" + 0.033*"uber" + 0.032*"mehr" + 0.026*"wirtz" + 0.024*"bundesliga" + 0.000*"frimpong" + 0.000*"ball" + 0.000*"hinspiel" + 0.000*"adli" + 0.000*"union" + 0.000*"hlozek" + 0.000*"diabi"'),
 (1,
  '0.176*"tor" + 0.117*"diabi" + 0.114*"ball" + 0.104*"nicht" + 0.101*"wirtz" + 0.082*"hlozek" + 0.080*"leverkusen" + 0.067*"mehr" + 0.062*"fur" + 0.058*"frimpong" + 0.037*"hradecki" + 0.000*"trainer" + 0.000*"alonso" + 0.000*"uber" + 0.000*"europaleagu" + 0.000*"sagt" + 0.000*"hinspiel" + 0.000*"union" + 0.000*"spiel" + 0.000*"adli"'),
 (2,
  '0.139*"uber" + 0.139*"adli" + 0.123*"fur" + 0.089*"frimpong" + 0.081*"trainer" + 0.062*"diabi" + 0.060*"alonso" + 0.058*"hlozek" + 0.054*"nicht" + 0.051*"wirtz" + 0.050*"tor" + 0.045*"ball" + 0.025*"bundesliga" + 0.024*"hradecki" + 0.000*"europaleagu" + 0.000*"leverkusen" + 0.000*"sagt" + 0.000*"

In [173]:
pyLDAvis.enable_notebook()
LDAvis_prepared = prepare(models_arr['cluster_4'], corpus, dictionary)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.043011 -0.219534       1        1  41.618912
2      0.117218  0.030767       2        1  23.421633
1      0.212454  0.130202       3        1  22.688882
3     -0.372683  0.058565       4        1  12.270573, topic_info=           Term        Freq       Total Category  logprob  loglift
11        union  211.000000  211.000000  Default  21.0000  21.0000
7      hinspiel  161.000000  161.000000  Default  20.0000  20.0000
6   europaleagu  242.000000  242.000000  Default  19.0000  19.0000
2           tor  286.000000  286.000000  Default  18.0000  18.0000
13         adli  181.000000  181.000000  Default  17.0000  17.0000
..          ...         ...         ...      ...      ...      ...
3       trainer    0.328982  268.944258   Topic4  -7.6341  -4.6083
8         spiel    0.330673  288.718445   Topic4  -7.6290  -4.6741
2           tor    0.325751  286.147765   Topic4  -7.6439  -4.6802
5         nicht    0.330541  543.366885   Topic4  -7.6293  -5.3068
4           fur    0.330032  689.859404   Topic4  -7.6309  -5.5471

[105 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
13        2  0.993217         adli
0         1  0.781676       alonso
0         2  0.189350       alonso
0         4  0.026703       alonso
15        2  0.286122         ball
15        3  0.710372         ball
1         1  0.466421   bundesliga
1         2  0.274855   bundesliga
1         4  0.258198   bundesliga
16        2  0.354171        diabi
16        3  0.642755        diabi
6         1  0.499236  europaleagu
6         4  0.499236  europaleagu
9         2  0.614261     frimpong
9         3  0.386561     frimpong
4         1  0.655206          fur
4         2  0.231931          fur
4         3  0.113067          fur
7         4  0.994632     hinspiel
14        2  0.420130       hlozek
14        3  0.576978       hlozek
18        1  0.540065     hradecki
18        2  0.183978     hradecki
18        3  0.278935     hradecki
20        1  0.575329   leverkusen
20        3  0.421074   leverkusen
10        1  0.461733         mehr
10        3  0.531310         mehr
5         1  0.631249        nicht
5         2  0.128826        nicht
5         3  0.239249        nicht
12        1  0.990396         sagt
8         1  0.997512        spiel
2         2  0.227155          tor
2         3  0.772328          tor
3         1  0.609792      trainer
3         2  0.390415      trainer
17        1  0.242847         uber
17        2  0.582833         uber
17        4  0.171612         uber
11        4  0.996459        union
19        1  0.173789        wirtz
19        2  0.194408        wirtz
19        3  0.374088        wirtz
19        4  0.259211        wirtz, R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])

# Examining the model with 8 clusters

In [167]:
# Create an empty array to store the most likely topics
most_likely_topics = []

#loop over all documents within the corpus
for doc in corpus:
    
    # predict the topics for the documents using the model with 8 clusters
    topics = models_arr['cluster_8'].get_document_topics(doc)

    # Get the most likely topic for the document
    most_likely_topic = max(topics, key=lambda x: x[1])
    
    # Append the most likely topic ID to the array
    most_likely_topics.append(most_likely_topic[0])

# Print the array of most likely topics
print(most_likely_topics)


[5, 0, 6, 0, 7, 3, 2, 7, 0, 6, 7, 1, 5, 7, 5, 6, 7, 5, 2, 0, 7, 7, 0, 0, 1, 0, 1, 0, 4, 0, 6, 0, 2, 7, 0, 6, 6, 0, 6, 6, 6, 0, 0, 0, 6, 0, 0, 6, 3, 5, 0, 5, 5, 4, 5, 5, 0, 5, 7, 6, 4, 0, 7, 3, 0, 3, 6, 7, 6, 0, 6, 0, 0, 0, 7, 0, 0, 0, 0, 0, 7, 7, 4, 0, 6, 0, 7, 7, 7, 7, 0, 1, 6, 0, 0, 5, 0, 1, 6, 5, 5, 0, 0, 0, 7, 1, 5, 0, 5, 7, 2, 7, 7, 0, 5, 0, 7, 0, 5, 2, 0, 0, 6, 3, 3, 5, 3, 0, 0, 4, 1, 3, 3, 3, 5, 1, 5, 0, 0, 5, 5, 5, 5, 0, 5, 5, 3, 5, 5, 4, 5, 5, 5, 2, 5, 5, 5, 5, 7, 0, 5, 5, 5, 4, 6, 0, 5, 0, 0, 5, 3, 6, 1, 6, 0, 0, 6, 5, 0, 0, 3, 5, 0, 2, 7, 6, 2, 7, 5, 5, 0, 6, 5, 0, 0, 7, 7, 5, 5, 0, 6, 0, 5, 4, 6, 5, 7, 0, 0, 7, 6, 5, 4, 0, 5, 5, 5, 5, 0, 0, 0, 0, 7, 5, 2, 1, 7, 0, 7, 0, 1, 6, 0, 0, 5, 7, 7, 0, 1, 3, 3, 0, 0, 4, 3, 3, 3, 7, 3, 6, 5, 5, 5, 0, 6, 5, 0, 3, 0, 6, 5, 5, 4, 5, 5, 5, 5, 4, 0, 3, 6, 3, 6, 0, 0, 0, 0, 4, 6, 0, 0, 0, 0, 3, 0, 0, 6, 0, 7, 7, 7, 0, 0, 4, 4, 6, 0, 7, 0, 7, 6, 0, 0, 0, 0, 6, 6, 7, 5, 5, 5, 6, 5, 7, 5, 5, 5, 5, 5, 5, 0, 5, 0, 6, 0, 0, 5, 7, 5, 0, 7, 6, 7, 

In [168]:
# create a new column to append the predicted clusters
df['predicted_cluster'] = most_likely_topics

# show countplot
df["predicted_cluster"].value_counts()


predicted_cluster
0    182
5    162
7     91
6     72
3     44
4     27
2     25
1     19
Name: count, dtype: int64

# Cluster naming 
0 = europa league
___
1 = other
___
2 = Bundesliga infos
___
3 = Bundesliga infos
___
4 = international_football_news
___
5 = player_performance
___
6 = Game descriptions
___
7 = Eurpoa League


In [169]:
def map_cluster_topic(cluster):
    if cluster == 0 or cluster == 7:
        return 'Europa League'
    elif cluster == 1:
        return 'Other'
    elif cluster == 2 or cluster == 3:
        return 'Bundesliga News'
    elif cluster == 4:
        return 'International Football News'
    elif cluster == 5:
        return 'Player Performance'
    else:
        return 'Game description'

# Apply the function and create 'sentiment_tb_desc' column
df['topic'] = np.vectorize(map_cluster_topic)(df['predicted_cluster'])

In [170]:
df.to_csv('/Users/kevingiesen/Library/Mobile Documents/com~apple~CloudDocs/BIPM Master/Semester 2/TWSM/TWSM Project/SS23-BIPM-Analytics-Lab---Group-4-repository/Modeling/clustered_data_ger_v1.csv', index=False)

In [171]:
pd.set_option('display.max_colwidth', None)
df.head(200)

,data,player,language,publishedAt,predicted_cluster,topic
0,trainer alonso mitgereisten fans monaco glucksmoment hebt tor euro highlight geht bundesliga sonntag uhr freiburg weiter,exequiel palacios,de,2023-02-24T09:33:31Z,5,Player Performance
1,sechser droht viereinhalb monaten pause bitterer abschied dafur erscheint verletzungspause lang zudem konkurrenzsituation doppelsechs seit woche voll saft stehenden amiri demirbay andrich voraussetzungen extrem schwierige fur aranguiz zuletzt aufgrund personellen engpasse innenverteidigung zentral abwehrkette eingesetzte andrich kunftig mittelfeld auflaufen durfte alonso vorrucken jahrigen sechs sonntag hertha bsc nicht sicher einstufen mochte,exequiel palacios,de,2023-03-03T21:35:13Z,0,Europa League
2,sechser erklart budapest spiel jungsten fortschritt donnerstag achtelfinal hinspiel europaleague ferencvaros budapest empfangt andrich fehlen sah defensive mittelfeldspieler erfolg elfmeterschiessen monaco dritte gelbe karte gesperrt massive schwachung fur werkself doppelsechs andrich zuletzt erfolgsfaktor andrich ausgerechnet kurzen phase saison gleichzeitig spielerisch uberzeugt erfolgreichen fussball spielt zwangspause einlegen muss wurmt bedingt das argerlich irgendwann ware sperre wahrscheinlich gekommen daher lieber jetzt eventuell viertel halbfinale keine sorgen erklart jahrige lacheln,exequiel palacios,de,2023-03-07T11:34:39Z,6,Game description
3,bakker mittwoch abschluss training nicht ausgeschlossen jedoch bakker ungarn startelf steht option erklart alonso mittelfeld doppel sechs verzichten muss andrich gesperrt fur stehen demirbay amiri bereit frimpong budapest einsatzbereit hertha wegen muskularer probleme bereits minuten platz verlassen mussen gut trainiert fuhlt besser dabei erklart alonso grossen respekt ungarn hat das sehr guter gegner sehe niveau monaco,exequiel palacios,de,2023-03-08T14:25:18Z,0,Europa League
4,aktuelle nachrichten politik wirtschaft sport kultur allerdings andrich argentinischen weltmeister beide eigentlich gesetzten sechser gesperrt charles aranguiz zudem verletzt donnerstag machten sache zumindest ordentlich mehr thema,exequiel palacios,de,2023-03-09T19:53:46Z,7,Europa League
...,...,...,...,...,...,...
195,wirtz bisschen gala form vergangenen wochen entfernt gute fur verhaltnisse beinahe euphorische laune uberzeugend gespielten viertelfinale herrschte furs erste verweht roma sah aus seien elf fantasie frische abhandengekommen einflussreichsten spieler wirken erschopft wirtz startete stark verblasste starker diaby lichtgeschwinde duett rechten seite traten kaum erscheinung hradecky mitbekommen hatte mussen verbessern sagte unuberhorbaren seufzen wir wussten nicht leicht knacken sind mittel wege mussen finden gelingt nicht droht schlimmste bestrafungen nicht sportliche rauswurf wettbewerb boser stempel kompletten mission nachlassen drauf ankommt wurde altbekannte stigma klub vizekusen wachrufen nichts hassen mehr unglucklichen halbfinal hinspiel rom beginn gefahrlichere team gerat plan trainer strategen alonso duell mourinho durcheinander hoffnung aufs finale besteht trotzdem,jeremie frimpong,de,2023-05-12T10:25:02Z,7,Europa League
196,wehrt europaleague lange gut schwere hypothek fur ruckspiel wechsel machte roma machtig druck muhe spiel beruhigen nicht recht gelang prompt fiel edmond tapsoba andrich ball nicht wegbekamen hradecky zunachst tammy abraham parierte bove nachschuss erfolgreich war kurz schluss kam grossen chance ausgleich roma verteidiger bryan cristante rettete brust kurz torlinie fur geschlagenen torhuter juventus turin sevilla schiedsrichter siebert berlin zuschauer ausverkauft tore nesyri gatti weniger sorgen verantwortlichen aussenbahnspieler halbzeit abbekommen schluss wieder sagte rolfes bei wahrscheinlich nicht problematisch,jeremie frimpong,de,2023-05-12T10:02:12Z,7,Europa League
197,vergibt leichtfertig gute ausgangsposition werksklub nicht fahrlassig zumindest leichtfertig gute ausgangsposition fur ruckspiel kommenden woche ve